In [49]:
import xarray as xr
import numpy as np
import os
import arrow
import json
import time
import pandas as pd
import datetime
import metpy.calc as mpcalc
from metpy.units import units

In [3]:
timeStrList = ['000', '003', '006', '009', '012', '015', '018', '021', '024', '027', '030', '033', '036', '039', '042', '045', '048', '051', '054', '057', '060', '063', '066', '069', '072', '078', '084', '090',
               '096', '102', '108', '114', '120', '126', '132', '138', '144', '150', '156', '162', '168', '174', '180', '186', '192', '198', '204', '210', '216', '222', '228', '234', '240', ]
fcHour_list = list(range(0, 72+1, 3)) + list(range(78, 240+1, 6))

config = {
    'sstk':{
        'name':'sstk',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'sea_surface_skin_temperature',
        'units': 'K',
        'long_name': 'sea surface temperature',
        'short_name': 'SST',
    },
    'visi':{
        'name':'visi',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'visibility_in_air',
        'units': 'm',
        'long_name': 'visibility',
        'short_name': 'visibility',
    },
    't2md':{
        'name':'t2md',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'dew_point_temperature',
        'units': 'K',
        'long_name': 'dew point',
        'short_name': 'Td',
    },
    't2mm':{
        'name':'t2mm',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'air_temperature',
        'units': 'K',
        'long_name': 'air temperature in 2 metre',
        'short_name': 'T2m',
    },
    'sktk':{
        'name':'sktk',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'surface_temperature',
        'units': 'K',
        'long_name': 'surface temperature',
        'short_name': 'sktk',
    },
    'mn2t':{
        'name':'mn2t',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'air_temperature',
        'units': 'K',
        'long_name': 'air temperature minimum in 2 metre since 6 hr before',
        'short_name': 'T2m',
    },
    'mx2t':{
        'name':'mx2t',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'air_temperature',
        'units': 'K',
        'long_name': 'air temperature maxium in 2 metre since 6 hr before',
        'short_name': 'T2m',
    },
    'rhum':{
        'name':'rhum',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'relative_humidity',
        'units': '0.01',
        'long_name': 'relative_humidity',
        'short_name': 'RH',
    },
    'temp':{
        'name':'temp',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'valid_min': 0.0,
        'standard_name': 'air_temperature',
        'units': 'K',
        'long_name': 'air temperature',
        'short_name': 'Temp',
    },
    'uwnd':{
        'name':'uwnd',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'standard_name': 'eastward_wind',
        'units': 'm/s',
        'long_name': 'u wind',
        'short_name': 'Uwnd',
    },
    'vwnd':{
        'name':'vwnd',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'standard_name': 'northward_wind',
        'units': 'm/s',
        'long_name': 'v wind',
        'short_name': 'Vwnd',
    },
    'u10m':{
        'name':'u10m',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'standard_name': 'eastward_wind',
        'units': 'm/s',
        'long_name': 'u wind',
        'short_name': 'Uwnd',
    },
    'v10m':{
        'name':'v10m',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'standard_name': 'northward_wind',
        'units': 'm/s',
        'long_name': 'v wind',
        'short_name': 'Vwnd',
    },
    'u100':{
        'name':'u100',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'standard_name': 'eastward_wind',
        'units': 'm/s',
        'long_name': 'u wind',
        'short_name': 'Uwnd',
    },
    'v100':{
        'name':'v100',
        'missing_value': -999.9,
        # '_FillValue':  -999.9,
        'standard_name': 'northward_wind',
        'units': 'm/s',
        'long_name': 'v wind',
        'short_name': 'Vwnd',
    },
}

In [4]:
dir_path = 'H:/data/ec_thin/'
var_symbols = ['t2md',
't2mm',
'sstk',
'u100',
'v100',
'u10m',
'v10m',
'rhum',
'temp',]

ds_list = []
for iSymbol in var_symbols:
    i_dataset = xr.open_dataset(f'{dir_path}{iSymbol}.nc')
    ds_list.append(i_dataset)



In [26]:
area = [105, 125, 15, 28]
da_test = ds_list[3]['u100024']
# print(da_test)
da_test.sel(time = da_test.time[0].values)

<xarray.DataArray 'u100024' (time: 62, level: 1, lat: 561, lon: 2880)>
[100172160 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 0.0 0.125 0.25 0.375 0.5 ... 359.5 359.6 359.8 359.9
  * lat      (lat) float32 -10.0 -9.875 -9.75 -9.625 ... 59.62 59.75 59.88 60.0
  * level    (level) float32 0.0
  * time     (time) datetime64[ns] 2022-10-01 2022-10-01T12:00:00 ... 2022-10-01


<xarray.DataArray 'u100024' (time: 36, level: 1, lat: 561, lon: 2880)>
[58164480 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 0.0 0.125 0.25 0.375 0.5 ... 359.5 359.6 359.8 359.9
  * lat      (lat) float32 -10.0 -9.875 -9.75 -9.625 ... 59.62 59.75 59.88 60.0
  * level    (level) float32 0.0
  * time     (time) datetime64[ns] 2022-10-01 2022-10-01 ... 2022-10-01

numpy.datetime64('2022-10-01T00:00:00.000000000')

In [27]:
time_step = '024'
area = [105, 125, 15, 28]
da_list = []
for iSymbol, i_ds in zip(var_symbols, ds_list):
    i_dataArray = i_ds[f'{iSymbol}{time_step}']
    print(iSymbol)
    if iSymbol == 'rhum' or iSymbol == 'temp':
        sud_dataArray = i_dataArray.sel(time=i_dataArray.time[0].values, level=[1000.0, 925.0], lat=slice(area[2], area[3]), lon=slice(area[0], area[1]))
    else:
        sud_dataArray = i_dataArray.sel(time=i_dataArray.time[0].values, level=0.0, lat=slice(area[2], area[3]), lon=slice(area[0], area[1]))
    da_list.append(sud_dataArray)



t2md
t2mm
sstk
u100
v100
u10m
v10m
rhum
temp


In [28]:
(t2md, t2mm,sstk,u100,v100,u10m,v10m,rhum,temp) = da_list

In [30]:
print(rhum)

<xarray.DataArray 'rhum024' (level: 2, lat: 53, lon: 81)>
array([[[96.049126, 92.549126, ..., 79.049126, 75.049126],
        [94.549126, 92.549126, ..., 78.549126, 77.549126],
        ...,
        [83.049126, 80.549126, ..., 84.049126, 81.549126],
        [93.549126, 87.049126, ..., 84.549126, 81.049126]],

       [[99.498184, 98.498184, ..., 93.498184, 92.998184],
        [99.498184, 98.498184, ..., 93.498184, 93.998184],
        ...,
        [82.998184, 80.998184, ..., 83.998184, 83.498184],
        [93.498184, 86.498184, ..., 87.498184, 83.998184]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 105.0 105.2 105.5 105.8 ... 124.2 124.5 124.8 125.0
  * lat      (lat) float32 15.0 15.25 15.5 15.75 16.0 ... 27.25 27.5 27.75 28.0
  * level    (level) float32 1e+03 925.0
    time     datetime64[ns] 2022-10-01


In [37]:
t2md.time.dt.dayofyear.values

array(274, dtype=int64)

In [48]:


iTime = t2md.time.dt
year_sin = np.sin((iTime.dayofyear.item() / 365.25) * 2 * np.pi)
year_cos = np.cos((iTime.dayofyear.item() / 365.25) * 2 * np.pi)
day_sin = np.sin((iTime.hour.item() / 24) * 2 * np.pi)
day_cos = np.cos((iTime.hour.item() / 24) * 2 * np.pi)
print(year_sin, year_cos, day_sin, day_cos)

-0.9999994220246925 0.001075151282798226 0.0 1.0


In [72]:
temp.shape

(2, 53, 81)

In [92]:
rhum_unit = np.clip(rhum, 0, 100)*units.percent
temp_unit = temp*units.kelvin
td_upper = mpcalc.dewpoint_from_relative_humidity(temp_unit, rhum_unit)

_x, level_mesh, _z = np.meshgrid(temp.lat, temp.level,temp.lon)
level_mesh = np.array(level_mesh)*units.hPa
theta_e = mpcalc.equivalent_potential_temperature(level_mesh, temp_unit, td_upper)
theta = mpcalc.potential_temperature(level_mesh, temp_unit)
theta_e = theta_e.metpy.convert_units('degC')
theta = theta.metpy.convert_units('degC')
print(theta_e)


<xarray.DataArray (level: 2, lat: 53, lon: 81)>
<Quantity([[[78.6026   78.53525  79.74249  ... 81.50897  81.95941  81.10321 ]
  [79.08856  78.24948  78.58316  ... 81.245636 81.74356  80.887085]
  [78.64548  78.0004   78.74225  ... 80.9487   81.429535 80.739685]
  ...
  [93.39929  91.862274 89.30914  ... 76.8248   77.14114  76.16571 ]
  [93.73312  90.188324 88.94571  ... 77.113434 76.935486 75.89676 ]
  [91.90082  89.29764  89.007355 ... 77.29501  76.701935 75.579865]]

 [[78.46161  77.17966  77.454315 ... 78.89252  79.664185 80.07892 ]
  [78.46161  77.72977  77.618805 ... 79.733765 79.664185 80.29477 ]
  [78.37463  78.39194  77.89398  ... 80.015015 80.08453  80.57669 ]
  ...
  [85.40427  83.40671  81.4205   ... 67.82391  68.460785 68.35693 ]
  [85.238434 82.50644  81.27893  ... 67.40573  68.6651   68.239136]
  [82.92325  80.85257  80.90384  ... 68.948456 69.74277  68.310394]]], 'degree_Celsius')>
Coordinates:
  * lon      (lon) float32 105.0 105.2 105.5 105.8 ... 124.2 124.5 124.8 125.

In [ ]:
# TODO apply_ufunc测试 https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html